In [23]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2

In [24]:
import flax.nnx as nn
import jax.numpy as jnp
from jax_gpt2 import CausalSelfAttention, GPTConfig

In [25]:
csa = CausalSelfAttention(GPTConfig(), nn.Rngs(0))

x = jnp.ones([1,3,768])
csa.c_attn(x).shape

qkv = csa.c_attn(x)
# jnp.split(qkv, csa.n_embd, -1)
[x.shape for x in jnp.split(qkv, [csa.n_embd, csa.n_embd*2], -1)]

(1, 3, 2304)

[(1, 3, 768), (1, 3, 768), (1, 3, 768)]

In [26]:
[x.shape for x in jnp.array_split(qkv, 3, axis=2)]

[(1, 3, 768), (1, 3, 768), (1, 3, 768)]

In [34]:
s = jnp.split(qkv, [csa.n_embd, csa.n_embd*2], -1)
ars = jnp.array_split(qkv, 3, axis=2)
jnp.allclose(s[0], ars[0])
jnp.allclose(s[1], ars[1])
jnp.allclose(s[2], ars[2])

Array(True, dtype=bool)

Array(True, dtype=bool)

Array(True, dtype=bool)

In [28]:
q, k, v = jnp.split(qkv, [csa.n_embd, csa.n_embd*2], -1)
B, T, C = x.shape
# q = q.view().reshape(B, T, csa.n_head, C // csa.n_head).transpose(1, 2) # (B, nh, T, hs)

q.view().reshape(B, T, csa.n_head, C // csa.n_head).shape
q.view().reshape(B, T, csa.n_head, C // csa.n_head).transpose([0,2,1,3]).shape

(1, 3, 12, 64)

(1, 12, 3, 64)

In [29]:
jnp.allclose(
    q.view().reshape(B, T, csa.n_head, C // csa.n_head), 
    q.reshape(B, T, csa.n_head, C // csa.n_head)
    )

Array(True, dtype=bool)

In [30]:
jnp.allclose(
    q.reshape(B, T, csa.n_head, C // csa.n_head).swapaxes(1, 2),
    q.view().reshape(B, T, csa.n_head, C // csa.n_head).transpose([0,2,1,3])
)

Array(True, dtype=bool)

In [31]:
csa(x).shape

(1, 3, 768)

In [32]:
x = jnp.ones([5,8]) # (B, T)
nn.make_causal_mask(x).shape

(5, 1, 8, 8)